In [1]:
dirOut = r"C:\Users\juaco\OneDrive\Documentos\ArcGIS\Packages\DAG_1s2024_L_TRABAJO_1_PARTE_2_JBV_d59d9f\p30\dag_1s2024_l_trabajo_1_parte_2_jbv.gdb"

In [2]:
#Ordenar ALTURAS PROMEDIO DEL DEM A TABLAS POR MANZANAS
fcIn = "IND_RIESGO_MANZ_CODE"
zone_field = "MANZENT_I"
raster_value = "DEM_CONURBA"
outName = "DEM_MANZANAS_CODE"
fcOut = f"{dirOut}\{outName}"

arcpy.ia.ZonalStatisticsAsTable(fcIn, zone_field, raster_value, fcOut, statistics_type="MEAN")


<geoprocessing server result object object at 0x00000238A6E68C30>

In [1]:
arcpy.management.JoinField(
    in_data="IND_RIESGO_MANZ_CODE",
    in_field="MANZENT_I",
    join_table="DEM_MANZANAS_CODE",
    join_field="MANZENT_I",
    fields="MEAN",
    fm_option="NOT_USE_FM",
    field_mapping=None
)


#Ordenar OBTENCIÓN DE ALTURAS DEM PROMEDIO POR MANZANAS
#fcIn2 = "IND_RIESGO_MANZ_CODE"

#arcpy.management.AddJoin(fcIn2, zone_field, outName, zone_field)


<Result 'IND_RIESGO_MANZ_CODE'>

In [4]:
#Ordenar UNIÓN DE ESTABLECIMIENTOS DE SALUD Y EDUCACIÓN EN UNA SOLA CAPA
fcIn3 = "EST_SALUD_CONURB;EST_EDU_CONURB"
outName = "EST_SALUD_EDU_CODE"
fcOut = f"{dirOut}\CAPAS_RESULTADOS\{outName}"

arcpy.management.Merge(fcIn3, fcOut)


<Result 'C:\\Users\\juaco\\OneDrive\\Documentos\\ArcGIS\\Packages\\DAG_1s2024_L_TRABAJO_1_PARTE_2_JBV_d59d9f\\p30\\dag_1s2024_l_trabajo_1_parte_2_jbv.gdb\\CAPAS_RESULTADOS\\EST_SALUD_EDU_CODE'>

In [9]:
#Ordenar CALCULAR CANTIDAD DE ESTABLECIMIENTOS DE SALUD Y EDUCACIÓN HAY POR MANZANA (JOIN_COUNT)
fcIn4 = "IND_RIESGO_MANZ_CODE"
join_features = "EST_SALUD_EDU_CODE"
outName = "IND_RIESGO_MANZ_CODE_1"
fcOut = f"{dirOut}\CAPAS_RESULTADOS\{outName}"

arcpy.analysis.SpatialJoin(fcIn4, join_features, fcOut)


<Result 'C:\\Users\\juaco\\OneDrive\\Documentos\\ArcGIS\\Packages\\DAG_1s2024_L_TRABAJO_1_PARTE_2_JBV_d59d9f\\p30\\dag_1s2024_l_trabajo_1_parte_2_jbv.gdb\\CAPAS_RESULTADOS\\IND_RIESGO_MANZ_CODE_1'>

In [10]:
#Ordenar CREACIÓN DEL CAMPO DE DENSIDAD POBLACIONAL POR MANZANA
fcIn5 = "IND_RIESGO_MANZ_CODE_1"
nombre_campo = "DENS_POB_MANZ"
formula = "!TOTAL_PERS! / !AREA!"

arcpy.management.CalculateField(fcIn5, nombre_campo, formula, field_type="DOUBLE")

<Result 'IND_RIESGO_MANZ_CODE_1'>

In [11]:
fcIn6 = "IND_RIESGO_MANZ_CODE_1"
arcpy.analysis.Near(
    fcIn6,
    near_features="BORDECOSTERO",
    field_names="NEAR_FID DIST_BC_FID;NEAR_DIST DIST_BC",
    distance_unit="Meters"
)

arcpy.analysis.Near(
    fcIn6,
    near_features="VIASMAR_FINAL",
    field_names="NEAR_FID DIST_VM_FID;NEAR_DIST DIST_VM",
    distance_unit="Meters"
)

arcpy.analysis.Near(
    fcIn6,
    near_features="RED_HIDRO_CONURB",
    field_names="NEAR_FID DIST_RH_FID;NEAR_DIST DIST_RH",
    distance_unit="Meters"
)


<Result 'IND_RIESGO_MANZ_CODE_1'>

In [12]:
#Ordenar ELIMINACIÓN DE 0 DEL CAMPO DE DISTANCIAS A LAS VIAS QUE CONECTAN AL BORDE COSTERO
arcpy.management.SelectLayerByAttribute(
    fcIn6,
    selection_type="NEW_SELECTION",
    where_clause="DIST_VM = 0",
    invert_where_clause=None
)

arcpy.management.CalculateField(
    fcIn6,
    field="DIST_VM",
    expression="0.01",
)

arcpy.management.SelectLayerByAttribute(fcIn6, "CLEAR_SELECTION")

<Result 'IND_RIESGO_MANZ_CODE_1'>

In [27]:
#Ordenar CREACIÓN DEL CAMPO DEL ÍNDICE DE RIESGO DE INUNDACIÓN POR AUMENTO DE NIVEL DEL MAR POR MANZANA
nombre_campo = "IND_RIESGO"
formula2 = "((0.2 * 1 / !MEAN!) + (0.1 * !DENS_POB_MANZ!) + (0.1 * 1 / (!DIST_VM! + 0.01)) + (0.1 * !Join_Count!) + (0.1 * !DIST_RH!) + (0.4 * !DIST_BC!))/(1000 * 3.772302)"

arcpy.management.CalculateField(fcIn6, nombre_campo, formula2, field_type="DOUBLE")

<Result 'IND_RIESGO_MANZ_CODE_1'>